Source: https://github.com/BeeAlarmed/BeeAlarmed

# Imports

In [23]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds
import config as cfg

Constants

In [5]:
CHAN_DIM = -1


# Model

In [7]:
def build_varroa_branch(input_shape):
    """! Creates the branch that detects varroa mite infestations
    """
    tmp_layer= layers.experimental.preprocessing.Rescaling(1./255)(input_shape)
    tmp_layer= Conv2D(64, (4, 4), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(32, (3, 3), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(16, (3, 3), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Flatten()(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= Dense(1)(tmp_layer)
    tmp_layer= Activation("sigmoid", name="varroa_output")(tmp_layer)

    return tmp_layer


In [8]:
def build_pollen_branch(input_shape):
    """! Creates the branch that detects pollen packets
    """
    tmp_layer= layers.experimental.preprocessing.Rescaling(1./255)(input_shape)
    tmp_layer= Conv2D(32, (4, 4), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(16, (3, 3), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Flatten()(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= Dense(1)(tmp_layer)
    tmp_layer= Activation("sigmoid", name="pollen_output")(tmp_layer)

    return tmp_layer


In [9]:
def build_wasps_branch(input_shape):
    """! Creates the branch that detects wasps
    """
    tmp_layer= layers.experimental.preprocessing.Rescaling(1./255)(input_shape)
    tmp_layer= Conv2D(16, (4, 4), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(8, (3, 3), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Flatten()(tmp_layer)
    tmp_layer= BatchNormalization()(tmp_layer)
    tmp_layer= Dense(1)(tmp_layer)
    tmp_layer= Activation("sigmoid", name="wasps_output")(tmp_layer)

    return tmp_layer


In [10]:
def build_cooling_branch(input_shape):
    """! Created the branch that detects bees that are cooling the hive
    """
    tmp_layer= layers.experimental.preprocessing.Rescaling(1./255)(input_shape)
    tmp_layer= Conv2D(64, (2, 2), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(32, (3, 3), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Conv2D(16, (2, 2), padding="valid")(tmp_layer)
    tmp_layer= Activation("relu")(tmp_layer)
    tmp_layer= MaxPooling2D(pool_size=(2, 2))(tmp_layer)

    tmp_layer= Flatten()(tmp_layer)
    tmp_layer= BatchNormalization(axis=CHAN_DIM)(tmp_layer)
    tmp_layer= Dense(1)(tmp_layer)
    tmp_layer= Activation("sigmoid", name="cooling_output")(tmp_layer)

    return tmp_layer


In [19]:
def get_bee_model(img_height, img_width):
    """! Creates BeeModel and returns it
    """
    input_shape = (img_height, img_width, 3)
    inputs = Input(shape=input_shape, name="input")

    pollen_m = build_pollen_branch(inputs)
    varroa_m = build_varroa_branch(inputs)
    wasps_m = build_wasps_branch(inputs)
    cooling_m = build_cooling_branch(inputs)

    model = Model(
        inputs=inputs,
        outputs=[varroa_m, pollen_m, wasps_m, cooling_m],
        name="beenet")

    losses = {
            "varroa_output": tf.losses.BinaryCrossentropy(),
            "pollen_output": tf.losses.BinaryCrossentropy(),
            "wasps_output": tf.losses.BinaryCrossentropy(),
            "cooling_output": tf.losses.BinaryCrossentropy()
            }
    loss_weights = {
            "varroa_output": 1.0,
            "pollen_output": 1.0,
            "wasps_output": 1.0,
            "cooling_output": 1.0
            }

    opt = Adam(learning_rate=0.0005)

    model.compile(
         optimizer=opt,
         loss=losses,
         metrics=["accuracy"],
         loss_weights=loss_weights,
         )

    return model

In [20]:
model = get_bee_model(150, 75)

In [24]:
model.summary()

Model: "beenet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 150, 75, 3)]         0         []                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 150, 75, 3)           0         ['input[0][0]']               
                                                                                                  
 rescaling_11 (Rescaling)    (None, 150, 75, 3)           0         ['input[0][0]']               
                                                                                                  
 conv2d_22 (Conv2D)          (None, 147, 72, 64)          3136      ['rescaling_9[0][0]']         
                                                                                             

# Training

In [ ]:
train, val = tfds.load('bee_dataset/bee_dataset_150',
                   batch_size=11,
                   as_supervised=True,
                   split=["train[0%:50%]", "train[50%:100%]"])

In [ ]:
model.fit(
    train,
    validation_data=val,
    epochs=20,
    verbose=1,
    callbacks=[]
)

In [ ]:
model.save(cfg.MODEL_SAVE_PATH)